In [ ]:
# !pip install --upgrade tensorflow
!pip install tensorflow==2.15
!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [ ]:
# !pip install --upgrade transformers

In [ ]:
# !pip show tensorflow
# !pip show Keras

In [ ]:
import os
import tensorflow as tf
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from datasets import load_dataset, load_metric
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# Checkpoints and final model
checkpoint_dir = '/content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sts'
final_model_path = '/content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sts-final'

Mounted at /content/drive


In [ ]:
# https://colab.research.google.com/drive/1VoWyzlXZVR5sNygejktJ4Uguw3HIZckb?usp=sharing

# Load ELECTRA BASE model and tokenizer
model_name = "google/electra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = TFElectraForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Load and preprocess the GLUE STS dataset
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html
# https://huggingface.co/docs/datasets/en/process
# https://huggingface.co/datasets/nyu-mll/glue
dataset = load_dataset("glue", "stsb")
metric = load_metric("glue", "stsb")

encoded_dataset = dataset.map(lambda examples: tokenizer(examples["sentence1"],
                                                         examples["sentence2"],
                                                         truncation=True,
                                                         padding="max_length",
                                                         return_tensors="tf"),
                              batched=True)

# Convert to TensorFlow dataset
# https://huggingface.co/docs/datasets/v1.16.1/use_dataset.html
train_data = encoded_dataset["train"].to_tf_dataset(columns=["attention_mask", "input_ids", "token_type_ids"],
                                                    label_cols=["label"],
                                                    shuffle=True,
                                                    batch_size=32)

validation_data = encoded_dataset["validation"].to_tf_dataset(columns=["attention_mask", "input_ids", "token_type_ids"],
                                                              label_cols=["label"],
                                                              shuffle=False,
                                                              batch_size=32)

# Checkpoints
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'checkpoint_epoch_{epoch}.weights.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Hyperparameters from the ELECTRA paper https://arxiv.org/abs/2003.10555
# "Table 6: Fine-tune hyperparameters"
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,
                                     epsilon=1e-6,
                                     beta_1=0.9,
                                     beta_2=0.999)

# Compile
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

# Finetune
model.fit(train_data, validation_data=validation_data, epochs=3, callbacks=[checkpoint_callback])

# Save the final model
model.save_pretrained(final_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
180/180 [==============================] - ETA: 0s - loss: 1.1928 - mse: 1.1928
Epoch 1: val_loss improved from inf to 0.47703, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sts/checkpoint_epoch_1.weights.h5


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


180/180 [==============================] - 172s 540ms/step - loss: 1.1928 - mse: 1.1928 - val_loss: 0.4770 - val_mse: 0.4770
Epoch 2/3
180/180 [==============================] - ETA: 0s - loss: 0.3594 - mse: 0.3594
Epoch 2: val_loss improved from 0.47703 to 0.44774, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sts/checkpoint_epoch_2.weights.h5
180/180 [==============================] - 86s 480ms/step - loss: 0.3594 - mse: 0.3594 - val_loss: 0.4477 - val_mse: 0.4477
Epoch 3/3
180/180 [==============================] - ETA: 0s - loss: 0.2060 - mse: 0.2060
Epoch 3: val_loss improved from 0.44774 to 0.44554, saving model to /content/drive/My Drive/W266_Final_Project/Finetune_BASE_GLUE/electra-base-sts/checkpoint_epoch_3.weights.h5
180/180 [==============================] - 86s 479ms/step - loss: 0.2060 - mse: 0.2060 - val_loss: 0.4455 - val_mse: 0.4455


- ELECTRA BASE fine tuned on GLUE Semantic Textual Similarity to predict how semantically similar two sentences are on a 1-5 scale.
- Ran in Google Colab Pro+ on A100 GPU
  - System RAM 5.1GB
  - GPU RAM 32.5GB
  - Ran in about 6 minutes
  - Final model weights saved
  - loss: 0.2060 - mse: 0.2060 - val_loss: 0.4455 - val_mse: 0.4455
- Did not implement the the warmup and decay, and instead used a common learning rate of 5e-5
- Next step: Benchmark against GLUE STS